In [17]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
import pandas as pd
pd.set_option('mode.chained_assignment',None)
import numpy as np
import pickle

from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import f1_score

from src.model import SolutionModel

# Baseline

В качестве бейзлайна, предсказывается класс одного сэмпла (`глубина`, `SP`, `GR`)

In [19]:
with open('data/train.pkl', 'rb') as fd:
    data = pickle.load(fd)
df_list = []
for key in data.keys():
    df = data[key]
    df['well_id'] = key
    df_list.append(df)
data = pd.concat(df_list)
ts_data = data

In [20]:
well_ids = ts_data['well_id'].unique()
train_wells, test_wells = train_test_split(well_ids,  random_state=42, test_size=0.25)
train_df = data[data['well_id'].isin(train_wells)]
test_df = data[data['well_id'].isin(test_wells)]
X_train = train_df[['MD', 'SP_log', 'GR_log']]
y_train = train_df['Facie_code']
X_test = test_df[['MD', 'SP_log', 'GR_log']]
y_test = test_df['Facie_code']

сохраним тестовые скважины, они дальше пригодятся

In [21]:
with open('data/train.pkl', 'rb') as fd:
    data = pickle.load(fd)
eval_data = {}
splitted_train = {}
for well_id in test_wells:
    eval_data[well_id] = data[well_id].copy()
for well_id in train_wells:
    splitted_train[well_id] = data[well_id].copy()
with open('data/eval.pkl', 'wb') as fd:
    pickle.dump(eval_data, fd)
with open('data/train_s.pkl', 'wb') as fd:
    pickle.dump(splitted_train, fd)

In [22]:
model = CatBoostClassifier(
    iterations=500,
    loss_function='MultiClass',
    eval_metric='MultiClass',
    learning_rate=0.01,
    random_seed=42,
    verbose=False
)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(f'F1@8: {np.sort(f1_score(y_test, y_pred, average=None))[::-1][:8].mean()}')
print(f'F1: {np.sort(f1_score(y_test, y_pred, average=None)).mean()}')

F1@8: 0.24430947993443328
F1: 0.15113731675584477


# Предложенное решение

1. С помощью алгоритма PELT выделяются границы фаций (решается задача Change Point Detection) на `GR_log`
2. С помощью random undesampling метода удаляем 40% сегментов с классом `-1` из обучающей выборки.
3. Используя `MD`, `GR_log`, `SP_log` из каждого сегмента излвекаются следующие признаки: `G`, `S`, `P`, `Am`, `am`, `Kp`, `R`, `mean_SP`, `min_SP`, `max_SP` на которых далее обучается модель многоклассовой классификации. Пусть $a_i$ это последовательность GR логов для одного сегмента размера $n$ $d_i$ последовательность соответствующи им глубин, $A_i$ последовательность пиковых точек $a_i$ для одного сегмента (т.е. таких точек $a_i$, что $a_{i-1} < a_i > a_{i+1}$). Тогда данные признаки можно посчитать следующим образом:

$$
    G = \frac{\sum_{i=1}^{n} i a_i}{n \sum_{i=1}^{n} a_i} 
$$

$$
    S = \sqrt{\frac{1}{n} \sum_{i=1}^{n}(a_i - \overline{a})^2}
$$

$$
    P = \frac{max(a) - mean(SP)}{d_n - d_0}
$$

$$
    Am = \frac{1}{2} \left( mean(a) + median(a) \right)
$$

$$
    am = max(a) - min(a)
$$
где параметры $G$, $S$, $P$, $Am$, $am$ описывают форму кривой `GR_log`.

$$
    Kp = \frac{\sum_{i=1}^n (a_i - \overline{a}) \cdot (d_i - \overline{d})}{\sum_{i=1}^n (a_i - \overline{a})^2}
$$
где $Kp$ описывает наклон кривой.

$R = \frac{L}{n}$, где $L$ число пиковых точек

$meanSp$, $minSP$, $maxSP$ - среднее, минимальное, максимальное значение `SP_log` на сегменте.

4. С помощью градиентного бустинга на деревьях (CatBoost) решается задача многоклассовой классификации, где один сэмпл это сегмент.

In [30]:
model = SolutionModel()
model.fit(train_data_path='data/train_s.pkl')
model.eval(test_data_path='data/eval.pkl')

0:	learn: 2.7999334	total: 15.3ms	remaining: 26.7s
1:	learn: 2.7739003	total: 29.9ms	remaining: 26.1s
2:	learn: 2.7443062	total: 45.4ms	remaining: 26.5s
3:	learn: 2.7118345	total: 59.6ms	remaining: 26s
4:	learn: 2.6854359	total: 74ms	remaining: 25.8s
5:	learn: 2.6597478	total: 89.1ms	remaining: 25.9s
6:	learn: 2.6336358	total: 104ms	remaining: 25.8s
7:	learn: 2.6129843	total: 118ms	remaining: 25.8s
8:	learn: 2.5865640	total: 133ms	remaining: 25.7s
9:	learn: 2.5654860	total: 148ms	remaining: 25.7s
10:	learn: 2.5435666	total: 162ms	remaining: 25.7s
11:	learn: 2.5250426	total: 177ms	remaining: 25.6s
12:	learn: 2.5061298	total: 192ms	remaining: 25.7s
13:	learn: 2.4836826	total: 208ms	remaining: 25.8s
14:	learn: 2.4620843	total: 224ms	remaining: 25.9s
15:	learn: 2.4438155	total: 242ms	remaining: 26.2s
16:	learn: 2.4253328	total: 260ms	remaining: 26.5s
17:	learn: 2.4061473	total: 277ms	remaining: 26.7s
18:	learn: 2.3894889	total: 295ms	remaining: 26.9s
19:	learn: 2.3721635	total: 319ms	remai

если мы хотим сделать прогноз и сохранить ответ, то вызываем следующую команду

In [24]:
model.build_sollution(test_data_path='data/test.pkl', save_to_path='predict.pkl')